In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install scikit-learn
# %pip install lightgbm
# %pip install xgboost
# %pip install flask
# %pip install xlrd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import pickle
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron

In [ ]:
if not os.path.isdir("templates"):
    os.mkdir("templates")

if not os.path.isdir("models"):
    os.mkdir("models")

In [ ]:
details = pd.read_csv("garmin_edge_820/4557226804_ACTIVITY_data.csv", sep=";")
X = details[["distance[m]", "altitude[m]"]]
y = details["heart_rate[bpm]"]
model = Pipeline([
    ("scaler", StandardScaler()),
    ("model",  Perceptron())
])
model.fit(X, y)
with open("models/model.bin", "wb") as f:
    pickle.dump(model, f)

In [ ]:
%%writefile flsk.py
import os.path
import pickle
from flask import Flask, request, render_template

In [ ]:
%%writefile -a flsk.py

app = Flask(__name__)
app.debug = True

In [ ]:
%%writefile -a  flsk.py

@app.route("/")
def home():
    return render_template("index.html")

In [ ]:
%%writefile -a  flsk.py

@app.route("/predict")
def predict():
    inputs = [
        float(request.args["LIMIT_BAL"]),
        int(request.args["PAY_6"]),
        int(request.args["EDUCATION"])
    ]
    with app.open_resource("models/model.bin", "rb") as f:
        model = pickle.load(f)
    response = model.predict([inputs])[0]
    return render_template("predict.html", resp=response)

In [ ]:
%%writefile -a  flsk.py

if __name__ == '__main__':
  app.run()

In [ ]:
%%writefile templates/index.html
<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Bootstrap demo</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet"
    integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" crossorigin="anonymous">
</head>

<body class="d-flex flex-column vh-100" data-bs-theme="dark">
  <div class="card m-auto">
    <div class="card-body">
      <h5 class="card-title">Card title</h5>
      <form method="GET" action="{{ url_for("predict") }}">
        <div class="mb-3">
          <label for="exampleInputEmail1" class="form-label">Email address</label>
          <input name="LIMIT_BAL" class="form-control" id="exampleInputEmail1" aria-describedby="emailHelp">
        </div>
        <div class="mb-3">
          <label for="exampleInputPassword1" class="form-label">Password</label>
          <input name="PAY_6" class="form-control" id="exampleInputPassword1">
        </div>
        <div class="mb-3">
          <label for="exampleInputPassword1" class="form-label">Password</label>
          <input name="EDUCATION" class="form-control" id="exampleInputPassword1">
        </div>
        <button type="submit" class="btn btn-primary">Submit</button>
      </form>
    </div>
  </div>

</body>

</html>

In [ ]:
%%writefile templates/predict.html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Bootstrap demo</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" crossorigin="anonymous">
  </head>
  <body class="d-flex flex-column vh-100" data-bs-theme="dark">
  <div class="card m-auto">
    <div class="card-body">
      <h5 class="card-title">Card title</h5>
      <p>Predizione: <b>{{ resp }}</b></p>
    </div>
  </div>

</body>

</html>